In [ ]:
import gym
import robo_gym
from robo_gym.wrappers.exception_handling import ExceptionHandling

In [ ]:
target_machine_ip = 'localhost' # or other machine 'xxx.xxx.xxx.xxx'

# initialize environment
env = gym.make('CubeRoomOnNavigationStack-v0', ip=target_machine_ip, gui=True)
env = ExceptionHandling(env)

env.reset(**{'new_room': True, 'new_agent_pose': True})


In [25]:
import numpy as np
import copy

import robo_gym_server_modules.robot_server.client as rs_client
from robo_gym.envs.simulation_wrapper import Simulation
from robo_gym_server_modules.robot_server.grpc_msgs.python import robot_server_pb2

from robo_gym.envs.mir_nav.mir_nav import Mir100NavEnv

In [26]:
class TestSim(Simulation):
    cmd = "roslaunch task_on_nav_robot_server sim_robot_server.launch"
    def __init__(self, ip=None, lower_bound_port=None, upper_bound_port=None, gui=False, **kwargs):
        Simulation.__init__(self, self.cmd, ip, lower_bound_port, upper_bound_port, gui, **kwargs)
        rs_address=self.robot_server_ip
        self.client = rs_client.Client(rs_address)

ts = TestSim(ip='localhost', gui=True)
client = ts.client

map_size = 256

rs_state_a = np.zeros([(map_size**2)*2 + 17])

ignore_start = 1
map_state_len = (map_size**2)*2
ignore_len = map_state_len + 6
ignore_index = ignore_start + ignore_len

rs_state_a[0] = map_size
rs_state_a[ignore_index] = 1
rs_state_a[ignore_index+1] = 1
rs_state_a[ignore_index+2] = 10
rs_state_a[ignore_index+3] = 0.4
rs_state_a[ignore_index+4] = 0.2
rs_state_a[ignore_index+5] = 8
rs_state_a[ignore_index+6] = 36
rs_state_a[ignore_index+7] = 40
rs_state_a[ignore_index+8] = 0.8
rs_state_a[ignore_index+9] = 0.15

state_msg = robot_server_pb2.State(state=rs_state_a)

if not client.set_state_msg(state_msg):
    raise Exception('fuuu')

rs_state = np.array(client.get_state_msg().state)
print(rs_state)

Starting new Robot Server | Tentative 1
Successfully started Robot Server at localhost:55959
[256.          -1.          -1.         ...  -0.50029051  -2.03906608
   0.        ]


In [10]:
rs = copy.deepcopy(rs_state_a)

rs[ignore_index] = 0
rs[ignore_index+1] = 1
state_msg = robot_server_pb2.State(state=rs)
client.set_state_msg(state_msg)

True

In [29]:
rs_state[map_state_len+1:map_state_len+4]

array([-2.05473971,  2.04361677, -0.98313546])

In [15]:
client.send_action([0,0,np.pi/4])

True

In [28]:
rs_state = np.array(client.get_state_msg().state)

In [9]:
from PIL import Image

arr = np.array(rs_state[1:1+map_size**2])
arr[arr == 0] = 255
arr[arr == 100] = 0
arr[arr == -1] = 0
arr = np.array(arr, dtype=np.uint8)
unique, counts = np.unique(arr, return_counts=True)
print(dict(zip(unique, counts)))
imgg = Image.fromarray(arr.reshape((256,256)).T, 'L')
imgg.show()

ar = np.array(rs_state[2+map_size**2:1+(map_size**2)*2+1],dtype=np.uint8)
unique, counts = np.unique(ar, return_counts=True)
print(dict(zip(unique, counts)))
img = Image.fromarray(ar.reshape((256,256)).T, 'L')
img.show()

{0: 60200, 255: 5336}
{0: 49381, 255: 16155}


In [ ]:
import sys
print(sys.path)

In [8]:
import numpy as np

g = np.reshape([1,1,1], (3,1))
g.flatten(), g

(array([1, 1, 1]),
 array([[1],
        [1],
        [1]]))

256.0